In [1]:
#General
import re
import sys
import pprint
import seaborn as sb
import numpy   as np
import pandas  as pd
import random  as rnd

from bios    import read
from os.path import isfile
from copy    import deepcopy
from time    import time
from tqdm    import tqdm
from getdist import plots

from scipy.interpolate import interp1d
from scipy.stats       import multivariate_normal
from itertools         import repeat

from getdist import plots,loadMCSamples,MCSamples

from theory_code.distance_theory import TheoryCalcs

#Plotting
import matplotlib
import matplotlib.pyplot as plt

from matplotlib import rc

rc('text', usetex=True)
rc('font', family='serif')
matplotlib.rcParams.update({'font.size': 18})

red    = '#8e001c'
yellow = '#ffb302'

sidelegend = {'bbox_to_anchor': (1.04,0.5), 
              'loc': "center left",
              'frameon': False}
bottomlegend = {'bbox_to_anchor': (0.35,-0.2), 
                'loc': "center left",
                'frameon': False,
                'ncols': 3}

# Settings

In [2]:
DESI_LCDM = {'omegam': 0.2948,
             'H0': 68.53}

fiducial = {'H0': 68.53,
            'omegam': 0.2948,
            'ombh2': 0.02218,
            'epsilon0_EM': 0.,
            'epsilon0_GW': 0.,
            'omk': 0.,
            'mnu': 0.06,
            'nnu': 3.,
            'MB': -19.2435}

theory_settings = {'zmin': 0.001,
                   'zmax': 5.,
                   'Nz': 1000,
                   'zdrag': 1060}

plot_pars = ['epsilon0_EM','omegam','H0']

zplot = np.linspace(0.01,3,100)

BAO_data = './data/DESI'
SN_data  = './data/Pantheon'

In [3]:
results = {r'Pantheon+$M_B$': {'root': './old_chains/Pantheon_LCDM',
                               'sampler': 'MH',
                               'DDR_model': 'constant',
                               'color': 'cyan',
                               'Nchains': 4,
                               'filled': True},
           r'DESI': {'root': './old_chains/DESI_LCDM',
                         'sampler': 'MH',
                         'DDR_model': 'constant',
                         'Nchains': 4,
                         'color': red,
                         'filled': False},
           #r'DESI+Pantheon+$M_B$': {'root': './chains/DESI_Pantheon_LCDM',
           #                         'sampler': 'MH',
           #                         'color': 'purple',
           #                         'Nchains': 1,
           #                         'filled': True},
          }

# Analysis

In [4]:
def analyze_results(name,info):
    analysis = deepcopy(info)
    
    print('')
    print('\x1b[1;31m Analyzing {} \x1b[0m'.format(name))
    
    chain_info = deepcopy(info)
    if info['sampler'] == 'MH':
        sample = loadMCSamples(info['root'], settings={'ignore_rows': 0.3})
    
        if info['Nchains']>1:
            print('R-1({}) with {:.0f}% of points ignored = {:.3f}'.format(name,30,
                                                                           sample.getGelmanRubin()))
        else:
            print('Single chain, no R-1 computed. Trust Cobaya and hope for the best')

    elif info ['sampler'] == 'Nautilus':
        chain_info['outyaml'] = read(info['root']+'.params.yaml', file_type='yaml')

        primary_pars = {par: chain_info['outyaml']['params'][par]['latex']
                        for par in chain_info['outyaml']['params']
                        if type(chain_info['outyaml']['params'][par]) == dict and 
                        'prior' in chain_info['outyaml']['params'][par]}

        derived_pars = {par: chain_info['outyaml']['params'][par]['latex']
                        for par in chain_info['outyaml']['params']
                        if type(chain_info['outyaml']['params'][par]) == dict and 
                        'prior' not in chain_info['outyaml']['params'][par]}

        pars = primary_pars | derived_pars

        raw_chains = pd.read_csv(info['root']+'_chain.txt',sep='\s+',
                                 header=0)#,names=['weight','minuslogpost']+list(pars.keys()))
        raw_chains['Chain number'] = 1

        chain_info['Nautilus pars']  = pars
        chain_info['Sampled points'] = raw_chains
        chain_info['evidence']       = chain_info['outyaml']['evidence']
        
        sample=MCSamples(samples=chain_info['Sampled points'][list(chain_info['Nautilus pars'].keys())].values,
                         names=list(chain_info['Nautilus pars'].keys()),
                         labels=list(chain_info['Nautilus pars'].values()),label=name)

        sample.root = info['root']

        
    analysis['MCsamples'] = sample
    analysis['bounds']    = sample.getTable(limit=1).tableTex()
    
    all_pars     = sample.getParamNames().list()
    labels       = sample.getParamNames().labels()
    primary_pars = sample.getParamNames().getRunningNames()
    
    analysis['means'] = {par:val for par,val in zip(all_pars,sample.getMeans()) if par in fiducial}
    
    #MM: theory specific part
    for par in fiducial:
        if par not in analysis['means']:
            analysis['means'][par] = fiducial[par]
            
    case_settings = deepcopy(theory_settings)
    case_settings['DDR_model'] = info['DDR_model']
            
    analysis['theory'] = TheoryCalcs(case_settings,analysis['means'])
    ###############################
    
    
    print(analysis['bounds'])
    if 'evidence' in chain_info:
        print('')
        print('log(Z)={:3f}'.format(chain_info['evidence']))
                               
    all_pars = sample.getParamNames().list()
                               
    analysis['Sampled points']         = pd.DataFrame(sample.makeSingleSamples(),columns=all_pars)
    analysis['Sampled points']['Case'] = name
                    
    return analysis

In [5]:
analyzed_results = {name: analyze_results(name,resdict) for name,resdict in results.items()}


 Analyzing Pantheon+$M_B$ 
R-1(Pantheon+$M_B$) with 30% of points ignored = 0.011
\begin{tabular} { l  c}

 Parameter &  68\% limits\\
\hline
{\boldmath$\Omega_\mathrm{m}$} & $0.320^{+0.089}_{-0.13}    $\\

{\boldmath$\Omega_\mathrm{b} h^2$} & $0.02218\pm 0.00056        $\\

{\boldmath$H_0            $} & $73.5\pm 1.2               $\\

{\boldmath$\epsilon_0^\mathrm{b}$} & $0.020\pm 0.089            $\\

{\boldmath$M_B            $} & $-19.246\pm 0.034          $\\

$r_\mathrm{d}              $ & $142^{+11}_{-15}           $\\

$omegaL                    $ & $0.679^{+0.13}_{-0.089}    $\\

$-\log\pi                  $ & $-3.65^{+0.17}_{-0.95}     $\\

$-\log\pi_\mathrm{0}       $ & $-3.65^{+0.17}_{-0.95}     $\\

$\chi^2                    $ & $1038.3\pm 2.3             $\\

$\chi^2_\mathrm{SNLike}    $ & $1038.3\pm 2.3             $\\
\hline
\end{tabular}

 Analyzing DESI 
R-1(DESI) with 30% of points ignored = 0.011
\begin{tabular} { l  c}

 Parameter &  68\% limits\\
\hline
{\boldm

In [ ]:
g = plots.get_subplot_plotter(subplot_size=1,width_inch=12, scaling=False)
g.settings.figure_legend_frame = False
g.settings.axes_fontsize=20
g.settings.axes_labelsize=20
g.settings.legend_fontsize=20
g.settings.axis_marker_color = 'black'
g.settings.axis_marker_ls = '--'
g.settings.axis_marker_lw = 1
g.settings.axis_tick_x_rotation = 45
g.triangle_plot([result['MCsamples'] for result in analyzed_results.values()], plot_pars,
    filled=[result['filled'] for result in analyzed_results.values()],
    legend_loc='upper right',
    legend_labels=[result for result in analyzed_results.keys()],
    contour_colors=[result['color'] for result in analyzed_results.values()],
    contour_lws=2,
    markers={k:v for k,v in DESI_LCDM.items()})    
g.fig.align_ylabels()
g.fig.align_xlabels();

# Mean observables

## load data

In [ ]:
datalist = []
dataset_SN = np.load(SN_data+'.npy',allow_pickle=True).item()
datalist.append(pd.DataFrame.from_dict({r'$z$': dataset_SN['z'],
                                        r'$m_B(z)$': dataset_SN['value'],
                                        r'error': [np.sqrt(var) for var in np.diag(dataset_SN['covmat'])]}))

dataset_DH = np.load(BAO_data+'_DH.npy',allow_pickle=True).item()
datalist.append(pd.DataFrame.from_dict({r'$z$': dataset_DH['z'],
                                        r'$d_H(z)$': dataset_DH['value'],
                                        r'error': [np.sqrt(var) for var in dataset_DH['covmat']]}))

dataset_DM = np.load(BAO_data+'_DM.npy',allow_pickle=True).item()
datalist.append(pd.DataFrame.from_dict({r'$z$': dataset_DM['z'],
                                        r'$d_M(z)$': dataset_DM['value'],
                                        r'error': [np.sqrt(var) for var in dataset_DM['covmat']]}))

dataset_DV = np.load(BAO_data+'_DV.npy',allow_pickle=True).item()
datalist.append(pd.DataFrame.from_dict({r'$z$': dataset_DV['z'],
                                        r'$d_V(z)$': dataset_DV['value'],
                                        r'error': [np.sqrt(var) for var in np.diag(dataset_DV['covmat'])]}))

data = pd.concat(datalist,ignore_index=True)

## pack theory

In [ ]:
functions = {'DH': r'$d_H(z)$',
             'DL_EM': r'$d_L^{\rm em}(z)$',
             'DL_GW': r'$d_L^{\rm gw}(z)$',
             'DM': r'$d_M(z)$',
             'DV': r'$d_V(z)$',
             'eta_EM': r'$\eta^{\rm em}(z)$',
             'eta_GW': r'$\eta^{\rm gw}(z)$',
             'mB': r'$m_B(z)$'}

full_results = []

for name,results in analyzed_results.items():
    theory = results['theory']
    theory_df = pd.DataFrame.from_dict({r'$z$': zplot}|{label: getattr(theory,func)(zplot) 
                                                     for func,label in functions.items()})
    
    theory_df['Case']  = name
    theory_df['color'] = results['color']
    
    full_results.append(theory_df)
    
plots_table = pd.concat(full_results,ignore_index=True)

In [ ]:
for func in functions.values():
    plt.figure()
    sb.lineplot(plots_table,x=r'$z$',y=func,hue='Case',palette=list(plots_table['color'].unique()))
    if func in data.columns:
        dataplot = data[[r'$z$',func,'error']].dropna(axis='index')
        plt.errorbar(dataplot[r'$z$'],dataplot[func],yerr=dataplot['error'],
                     ls='',color='black',marker='*')